In [ ]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as st
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10, 6
import seaborn as sns
import sklearn
from sklearn.ensemble.forest import RandomForestRegressor
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
import operator 
import psycopg2
import math
import requests
import json
from flask import jsonify
import csv
from csv import writer
from statistics import mean
from plotly.subplots import make_subplots
import plotly.graph_objects as go

SELECTING AREA OF COUNTRY

In [ ]:
def getRegion(regID):
    try:
        
        connection = psycopg2.connect(user = "teamcoronosis",
                                  password = "p1jGQwEMANAGZ5b72xWh",
                                  host = "coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com",
                                  port = "5432",
                                  database = "postgres")
        cursor = connection.cursor()

       
    # Print PostgreSQL Connection properties
        print ( connection.get_dsn_parameters(),"\n")

    # Print PostgreSQL version
        cursor.execute("SELECT version();")
        record = cursor.fetchone()
        print("You are connected to - ", record,"\n")
    
        postgres_select_query = "select area from regions where alpha2 = %s"
       
        cursor.execute(postgres_select_query, (regID,))
        
        reg = cursor.fetchall()
        for r in reg :
            print(r[0])
            return r[0]
        #print(reg)

       # connection.commit()
       # count = cursor.rowcount
       # print (count, "Record inserted successfully into worldprediction table")

    except (Exception, psycopg2.Error) as error :
        if(connection):
            print("Error fetching data from PostgreSQL table", error)
        
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
            
    finally:
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
    
    #return reg[0]

COUNTRY TIMELINE

In [ ]:
url = 'http://coronavirus-tracker-api.herokuapp.com/all'
url_iso2 = 'https://restcountries.eu/rest/v2/'



response_iso2 = requests.get(url_iso2)
lst_iso2 = json.loads(response_iso2.text)

response = requests.get(url)
lst = json.loads(response.text)

dict_pop = {}
dict_iso3 = {}
dict_affected = {}
dict_deaths = {}
dict_recovered = {}

for i in lst_iso2:
    dict_pop[i['alpha2Code']] = i['population']
    dict_iso3[i['alpha2Code']] = i['alpha3Code']
    dict_affected[i['alpha2Code']] = {}
    dict_deaths[i['alpha2Code']] = {}
    dict_recovered[i['alpha2Code']] = {}
      

## INITIALIZATION ##

for i in lst['confirmed']['locations']:
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_affected[code][j] = 0

for i in lst['recovered']['locations']:
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_recovered[code][j] = 0

for i in lst['deaths']['locations']:
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_deaths[code][j] = 0         


## Summing up ##

for i in lst['deaths']['locations']:
    name = i['country']
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_deaths[code][j] = dict_deaths[code][j] + i['history'][j] 

for i in lst['recovered']['locations']:
    name = i['country']
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_recovered[code][j] = dict_recovered[code][j] + i['history'][j] 

for i in lst['confirmed']['locations']:
    name = i['country']
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_affected[code][j] = dict_affected[code][j] + i['history'][j]                         



with open('country_model_new.csv', 'w') as csv_file:
    csv_writer = writer(csv_file)
    headers = ['Country_name', 'Country_code_iso2', 'Country_code_iso3', 'date', 'Tot_affected', 'Tot_dead', 'Tot_recov', 'Population','Rec/affected','Rec/population']
    csv_writer.writerow(headers)
    for i in lst['confirmed']['locations']:
        name = i['country']
        code = i['country_code']
        if code in dict_iso3:
            for j in i['history']:
                if dict_affected[code][j]!=0:
                    entry = [name, code, dict_iso3[code], j, dict_affected[code][j], dict_deaths[code][j], dict_recovered[code][j], dict_pop[code],dict_recovered[code][j]/dict_affected[code][j],(dict_recovered[code][j]/dict_pop[code])*1000000]
                    csv_writer.writerow(entry)

                    
                    
                    
                    

WORLD TIMELINE

In [ ]:
response = requests.get("https://corona-api.com/timeline?fbclid=IwAR1jTS9DQOC-myWVIJb20qYGVbpEu0r-T587oow4kx8Z2CQO7-aXHfPM5BY")
print(response.status_code)

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

Datas = response.json()['data']
#del Datas[0]
#jprint(Datas)
Datas.sort(key = operator.itemgetter('date'))
print(Datas)

with open('world_Data.csv','w') as new_file:
        fieldnames = ['date','deaths','confirmed','recovered','CFR_1','CFR_2','wRec/affected','wRec/population', 'wRec/Area']
        
        csv_writer = csv.DictWriter(new_file, fieldnames= fieldnames)
        csv_writer.writeheader()
        
        for line in Datas:
            line['CFR_1'] = int(line['deaths']) / (int(line['recovered']) + int(line['deaths']))
            line['CFR_2'] = int(line['deaths']) / int(line['confirmed'])
            line['wRec/affected'] = line['recovered']/line['confirmed']
            line['wRec/population'] = ((line['recovered']/7594000000) * 1000000)
            line['wRec/Area'] = ((line['recovered']/132025199.412) * 100)
            del line['updated_at']
            del line['new_confirmed']
            del line['new_recovered']
            del line['new_deaths']
            del line['active']
            #print(line['CFR1'],line['CFR2'])
            #print(line)
            csv_writer.writerow(line)

In [14]:
worldset = pd.read_csv('world_Data.csv')
worldset.head()

,date,deaths,confirmed,recovered,CFR_1,CFR_2,wRec/affected,wRec/population,wRec/Area
0,2020-01-21,17,555,28,0.377778,0.030631,0.050450,0.003687,0.000021
1,2020-01-22,35,1208,58,0.376344,0.028974,0.048013,0.007638,0.000044
2,2020-01-23,44,1594,66,0.400000,0.027604,0.041405,0.008691,0.000050
3,2020-01-24,68,2379,75,0.475524,0.028583,0.031526,0.009876,0.000057
4,2020-01-25,98,3556,91,0.518519,0.027559,0.025591,0.011983,0.000069


In [15]:
countryset = pd.read_csv('country_model_new.csv')
countryset.head()
                         

,Country_name,Country_code_iso2,Country_code_iso3,date,Tot_affected,Tot_dead,Tot_recov,Population,Rec/affected,Rec/population
0,Afghanistan,AF,AFG,2/24/20,1,0,0,27657145,0.0,0.0
1,Afghanistan,AF,AFG,2/25/20,1,0,0,27657145,0.0,0.0
2,Afghanistan,AF,AFG,2/26/20,1,0,0,27657145,0.0,0.0
3,Afghanistan,AF,AFG,2/27/20,1,0,0,27657145,0.0,0.0
4,Afghanistan,AF,AFG,2/28/20,1,0,0,27657145,0.0,0.0


ONLY PREFERED COUNTRIES

In [108]:
"""fig, axes = plt.subplots(3, 1, sharex=True)
axes[0].plot(result['date'], result['wRec/affected'], color='green')
axes[0].plot(result['date'], result['Rec/affected'], color='orange')
axes[0].set_title('time vs recovered/affected')
axes[0].legend(['Global','Bangladesh'], prop={'size': 12}, bbox_to_anchor=(0.5, 1.02), ncol=6, fancybox=True, shadow=True)

axes[1].plot(result['date'], result['Rec/population'], color='orange')
axes[1].plot(result['date'], result['wRec/population'], color='green')
axes[1].set_title('time vs recovered/population')

axes[2].plot(result['date'], result['Rec/Area'], color='orange')
axes[2].plot(result['date'], result['wRec/Area'], color='green')
axes[2].set_title('time vs recovered/area')
#plot_acf(df.value.diff().diff().dropna(), ax=axes[2, 1])

plt.show()"""

"fig, axes = plt.subplots(3, 1, sharex=True)\naxes[0].plot(result['date'], result['wRec/affected'], color='green')\naxes[0].plot(result['date'], result['Rec/affected'], color='orange')\naxes[0].set_title('time vs recovered/affected')\naxes[0].legend(['Global','Bangladesh'], prop={'size': 12}, bbox_to_anchor=(0.5, 1.02), ncol=6, fancybox=True, shadow=True)\n\naxes[1].plot(result['date'], result['Rec/population'], color='orange')\naxes[1].plot(result['date'], result['wRec/population'], color='green')\naxes[1].set_title('time vs recovered/population')\n\naxes[2].plot(result['date'], result['Rec/Area'], color='orange')\naxes[2].plot(result['date'], result['wRec/Area'], color='green')\naxes[2].set_title('time vs recovered/area')\n#plot_acf(df.value.diff().diff().dropna(), ax=axes[2, 1])\n\nplt.show()"

# FOR ALL COUNTRIES

In [6]:
def datasets1(iso2):
    dset = countryset.loc[countryset['Country_code_iso2'] == '%s'% (iso2)]
    dset.head()
    
    
    print(iso2)
    area = getRegion(iso2)
    
    
        
        
    dset = dset.copy()


    dset['Rec/Area'] = (dset.iloc[:,6:7]/area) * 100

    dset.head()
    
    dset['date']= pd.to_datetime(dset['date'],infer_datetime_format=True)
    
    return dset
    

In [7]:

    Bdset = datasets1('BD')
    ukset = datasets1('GB')
    usset = datasets1('US')
    Chset = datasets1('CN')
    indset = datasets1('IN')
    idset = datasets1('ID')
    itset = datasets1('IT')
    ruset = datasets1('RU')
    spset = datasets1('ES')
    brset = datasets1('BR')
    
    Bdset.head()
    
    

BD
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

130170
PostgreSQL connection is closed
GB
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

241930
PostgreSQL connection is closed
US
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty':

,Country_name,Country_code_iso2,Country_code_iso3,date,Tot_affected,Tot_dead,Tot_recov,Population,Rec/affected,Rec/population,Rec/Area
1725,Bangladesh,BD,BGD,2020-03-08,3,0,0,161006790,0.0,0.0,0.0
1726,Bangladesh,BD,BGD,2020-03-09,3,0,0,161006790,0.0,0.0,0.0
1727,Bangladesh,BD,BGD,2020-03-10,3,0,0,161006790,0.0,0.0,0.0
1728,Bangladesh,BD,BGD,2020-03-11,3,0,0,161006790,0.0,0.0,0.0
1729,Bangladesh,BD,BGD,2020-03-12,3,0,0,161006790,0.0,0.0,0.0


In [11]:
Bdset.iloc[0,0]

'Bangladesh'

In [12]:
worldset['date'] = pd.to_datetime(worldset['date'],infer_datetime_format=True)
dfs = [Bdset,ukset,usset,Chset,indset,idset,itset,ruset,spset,brset]
result = pd.concat(dfs,keys=['Bangladesh', 'United kingdom','United states','China','India','Indonesia','Italy','Russia','Spain','Brazil'] )
print(result)
#result['Country_name'] = dfs['Country_name']
resultf = pd.merge(result, worldset,on='date', how ='left')
    

    #result = pd.merge(Bdset, worldset,on='date', how='left')
resultf.head()

                Country_name Country_code_iso2 Country_code_iso3       date  \
Bangladesh 1725   Bangladesh                BD               BGD 2020-03-08   
           1726   Bangladesh                BD               BGD 2020-03-09   
           1727   Bangladesh                BD               BGD 2020-03-10   
           1728   Bangladesh                BD               BGD 2020-03-11   
           1729   Bangladesh                BD               BGD 2020-03-12   
...                      ...               ...               ...        ...   
Brazil     2361       Brazil                BR               BRA 2020-05-09   
           2362       Brazil                BR               BRA 2020-05-10   
           2363       Brazil                BR               BRA 2020-05-11   
           2364       Brazil                BR               BRA 2020-05-12   
           2365       Brazil                BR               BRA 2020-05-13   

                 Tot_affected  Tot_dead  Tot_recov 

,Country_name,Country_code_iso2,Country_code_iso3,date,Tot_affected,Tot_dead,Tot_recov,Population,Rec/affected,Rec/population,Rec/Area,deaths,confirmed,recovered,CFR_1,CFR_2,wRec/affected,wRec/population,wRec/Area
0,Bangladesh,BD,BGD,2020-03-08,3,0,0,161006790,0.0,0.0,0.0,3796.0,108774.0,60609.0,0.058940,0.034898,0.557201,7.981169,0.045907
1,Bangladesh,BD,BGD,2020-03-09,3,0,0,161006790,0.0,0.0,0.0,3987.0,112077.0,62322.0,0.060128,0.035574,0.556064,8.206742,0.047205
2,Bangladesh,BD,BGD,2020-03-10,3,0,0,161006790,0.0,0.0,0.0,4252.0,117569.0,64270.0,0.062053,0.036166,0.546658,8.463260,0.048680
3,Bangladesh,BD,BGD,2020-03-11,3,0,0,161006790,0.0,0.0,0.0,4613.0,125252.0,66688.0,0.064698,0.036830,0.532431,8.781670,0.050512
4,Bangladesh,BD,BGD,2020-03-12,3,0,0,161006790,0.0,0.0,0.0,3810.0,109467.0,66720.0,0.054020,0.034805,0.609499,8.785884,0.050536


In [87]:
Bdset.head()

,Country_code_iso2,Country_code_iso3,date,Tot_affected,Tot_dead,Tot_recov,Population,Rec/affected,Rec/population,Rec/Area
Country_name,,,,,,,,,,
Bangladesh,BD,BGD,2020-03-08,3,0,0,161006790,0.0,0.0,123689.628947
Bangladesh,BD,BGD,2020-03-09,3,0,0,161006790,0.0,0.0,123689.628947
Bangladesh,BD,BGD,2020-03-10,3,0,0,161006790,0.0,0.0,123689.628947
Bangladesh,BD,BGD,2020-03-11,3,0,0,161006790,0.0,0.0,123689.628947
Bangladesh,BD,BGD,2020-03-12,3,0,0,161006790,0.0,0.0,123689.628947


In [35]:
def datasetss(iso2):
    Bdset = countryset.loc[countryset['Country_code_iso2'] == '%s'% (iso2)]
    Bdset.head()
    
    
    print(iso2)
    area = getRegion(iso2)
    
    
        
        
    Bdset= Bdset.copy()


    Bdset['Rec/Area'] = (Bdset.iloc[:,6:7]/area) * 100

    Bdset.head()
    
    Bdset['date']= pd.to_datetime(Bdset['date'],infer_datetime_format=True)
    Bdset.drop_duplicates(subset ="date", 
                     keep = 'first', inplace = True)
    worldset['date'] = pd.to_datetime(worldset['date'],infer_datetime_format=True)
    #print(Bdset.Country_name)

    #result = pd.merge(Bdset, worldset,on='date', how='left')
    #result.head()
    
    fig = make_subplots(rows= 3, cols = 1,subplot_titles=("date vs recovered/affected", "date vs recovered/population", "date vs recovered/area"))

    fig.update_layout(height=1000, width=1000,
                  title_text="Multiple Subplots with Titles",xaxis=dict(title='Date'),yaxis=dict(title='Recovery rate'))
    fig.add_trace(go.Scatter(x=worldset['date'], y=worldset['wRec/affected'] ,
                    mode='lines+markers',
                    name='Global',line=dict(color='green', width=2)), row=1, col=1)

    fig.add_trace(go.Scatter(x=Bdset['date'], y=Bdset['Rec/affected'] ,
                    mode='lines+markers',
                    name= '%s'%(Bdset.iloc[0,0]),line=dict(color='orange', width=2) ), row=1, col=1)


    fig.add_trace(go.Scatter(x=worldset['date'], y=worldset['wRec/population'] ,
                    mode='lines+markers',
                    name='Global',line=dict(color='green', width=2) ), row=2, col=1)

    fig.add_trace(go.Scatter(x=Bdset['date'], y=Bdset['Rec/population'] ,
                    mode='lines+markers',
                    name= '%s'%(iso2),line=dict(color='orange', width=2) ), row=2, col=1)


    fig.add_trace(go.Scatter(x=worldset['date'], y=worldset['wRec/Area'] ,
                    mode='lines+markers',
                    name='Global',line=dict(color='green', width=2) ), row=3, col=1)

    fig.add_trace(go.Scatter(x=Bdset['date'], y=Bdset['Rec/Area'] ,
                    mode='lines+markers',
                    name='%s'%(iso2),line=dict(color='orange', width=2) ), row=3, col=1)



    fig.show()

In [36]:
datasetss('BD')

BD
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

130170
PostgreSQL connection is closed


In [37]:
datasetss('GB')

GB
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

241930
PostgreSQL connection is closed


In [38]:
datasetss('IN')

IN
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

2973190
PostgreSQL connection is closed


In [39]:
datasetss('US')

US
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

9147420
PostgreSQL connection is closed


In [40]:
datasetss('BR')

BR
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

8358140
PostgreSQL connection is closed


In [41]:
datasetss('ES')

ES
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

499564
PostgreSQL connection is closed


In [42]:
datasetss('CN')

CN
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

9388210
PostgreSQL connection is closed


In [43]:
datasetss('IT')

IT
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

294140
PostgreSQL connection is closed
